# 2. 데이터 확인

In [1]:
import pandas as pd

movies = [
    ['user1', '연평대전', 5],
    ['user1', '7번방의 선물', 4],
    ['user1', '국제시장', 4],
    ['user2', '연평대전', 5],
    ['user2', '7번방의 선물', 3],
    ['user2', '국제시장', 4],
    ['user2', '명량', 5],
    ['user3', '국제시장', 3],
    ['user3', '명량', 3],
    ['user3', '국가대표', 5]
]

df_movies = pd.DataFrame(movies, columns=['userid', 'moviename','ratings'])
df_movies

,userid,moviename,ratings
0,user1,연평대전,5
1,user1,7번방의 선물,4
2,user1,국제시장,4
3,user2,연평대전,5
4,user2,7번방의 선물,3
5,user2,국제시장,4
6,user2,명량,5
7,user3,국제시장,3
8,user3,명량,3
9,user3,국가대표,5


> 위와 같이 영화별로 평점을 처리한 데이터가 존재한다. 이를 이용하여 협업 필터링을 이용한 추천 시스템을 만들어 보자

# 2. 데이터 가공 및 변환

## 2.1 데이터 가공

In [2]:
df_ratings_movies = df_movies.pivot_table('ratings', index='userid', columns='moviename')
df_ratings_movies

moviename,7번방의 선물,국가대표,국제시장,명량,연평대전
userid,,,,,
user1,4.0,NaN,4.0,NaN,5.0
user2,3.0,NaN,4.0,5.0,5.0
user3,NaN,5.0,3.0,3.0,NaN


>위와 같이 row는 영화 이름을 columns은 사용자 id를 갖는 dataFrame을 만들 수 있다.

## 2.2 데이터 변환
>위와 같이 가공한 데이터에서 평가가 되지 않는 부분에 대해서는 결측치로 나타나게 된다. 이를 해결하기 위해 결측치를 0으로 대처하자

In [3]:
df_ratings_movies = df_ratings_movies.fillna(0)
df_ratings_movies

moviename,7번방의 선물,국가대표,국제시장,명량,연평대전
userid,,,,,
user1,4.0,0.0,4.0,0.0,5.0
user2,3.0,0.0,4.0,5.0,5.0
user3,0.0,5.0,3.0,3.0,0.0


>NaN으로 값이 지정되어 있으면 유사도를 구할 수 없기 때문에 0으로 변환시켰다.

# 3. 영화 간 유사도 산출

## 3.1 사용자 기반 추천
> 유사도 산출을 위해 cosine_similarity()를 이용할 것이다. cosine_similarity()는 행을 기준으로 유사도를 산출하는 함수이다. 현재 dataframe의 행은 사용자ID로 이루어져 있어 유사도를 산출하게 되면 사용자의 유사도를 구할 수 있다.

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

user_sim = cosine_similarity(df_ratings_movies, df_ratings_movies)
user_sim

array([[1.        , 0.81060226, 0.24238715],
       [0.81060226, 1.        , 0.47544349],
       [0.24238715, 0.47544349, 1.        ]])

>위와 같이 코사인 유사도를 이용하여 사용자별 평점 유사도를 측정할 수 있다.

In [5]:
user_based_collab = pd.DataFrame(data=user_sim, index=df_ratings_movies.index, columns=df_ratings_movies.index)
user_based_collab

userid,user1,user2,user3
userid,,,
user1,1.000000,0.810602,0.242387
user2,0.810602,1.000000,0.475443
user3,0.242387,0.475443,1.000000


In [6]:
user_based_collab['user1'].sort_values(ascending=False)

userid
user1    1.000000
user2    0.810602
user3    0.242387
Name: user1, dtype: float64

>이를 간단하게 DataFrame으로 만들고 user1과 유사도가 높은 순으로 정렬하여 출력했다. user1과 영화적 취향이 가장 비슷한 사람은 'user2'이다. 따라서 user1에게 user2가 본 영화를 추천하면 좋을 것이다.

> user2와 영화적 취향이 비슷하다고 판단하여 user2가 본 영화의 정렬된 정보를 확인했다. 이를 간단하게 함수를 만들어 보자.

In [7]:
def user_based_recommend(data, userid):
    user_sim = cosine_similarity(data, data)
    user_based_collab = pd.DataFrame(data=user_sim, index=data.index, columns=data.index)
    users = user_based_collab[userid].sort_values(ascending=False)
    return data.loc[users.index[1]].sort_values(ascending=False)

>위와 같이 영화적 취향이 비슷한 사람의 영화를 보여주고 있다. 이번에는 사용자가 못본 영화를 추천하는 시스템을 만들어 보자.

In [8]:
user_based_recommend(df_ratings_movies, 'user1')

moviename
연평대전       5.0
명량         5.0
국제시장       4.0
7번방의 선물    3.0
국가대표       0.0
Name: user2, dtype: float64

In [9]:
df_ratings_movies_T = df_ratings_movies.transpose()
df_ratings_movies_T

userid,user1,user2,user3
moviename,,,
7번방의 선물,4.0,3.0,0.0
국가대표,0.0,0.0,5.0
국제시장,4.0,4.0,3.0
명량,0.0,5.0,3.0
연평대전,5.0,5.0,0.0


In [10]:
df_ratings_movies_T[df_ratings_movies_T['user1'] == 0]

userid,user1,user2,user3
moviename,,,
국가대표,0.0,0.0,5.0
명량,0.0,5.0,3.0


>안본 영화의 평점을 0으로 수정했음으로 사용자의 평점이 0과 같은 정보를 추출하면 된다. 하지만 현재 columns 정보가 영화 정보로 되어 있어 정보를 비교할 수 없다. transpose가 전치를 해 주는 함수이므로 이를 이용하여 치환하고 평점이 0인 값만 출력했다.

> 이렇게 구한 테이블에서 사용자와 취향이 비슷한 사용자의 정보 중 보지 않은 정보를 추천할 수도 있다. 이를 간단하게 함수로 만들어 보자.

##3.2 아이템 기반 추천
> 기존의 DataFrame은 사용자를 행으로 가지고 있다. 아이템 기반 추천을 위해 행을 영화 제목으로 변경하고 시스템을 구축해 보자.

>위와 같이 transpose() 함수를 이용하여 간단하게 행, 열을 치환할 수 있다.

>각 영화별 평점 유사도를 구한 결과이다. 

In [11]:
item_movies_sim = cosine_similarity(df_ratings_movies_T, df_ratings_movies_T)
item_movies_sim

array([[1.        , 0.        , 0.87457307, 0.51449576, 0.98994949],
       [0.        , 1.        , 0.46852129, 0.51449576, 0.        ],
       [0.87457307, 0.46852129, 1.        , 0.7767238 , 0.88345221],
       [0.51449576, 0.51449576, 0.7767238 , 1.        , 0.60633906],
       [0.98994949, 0.        , 0.88345221, 0.60633906, 1.        ]])

In [12]:
item_based_collab = pd.DataFrame(data=item_movies_sim, index=df_ratings_movies_T.index, columns=df_ratings_movies_T.index)
item_based_collab

moviename,7번방의 선물,국가대표,국제시장,명량,연평대전
moviename,,,,,
7번방의 선물,1.000000,0.000000,0.874573,0.514496,0.989949
국가대표,0.000000,1.000000,0.468521,0.514496,0.000000
국제시장,0.874573,0.468521,1.000000,0.776724,0.883452
명량,0.514496,0.514496,0.776724,1.000000,0.606339
연평대전,0.989949,0.000000,0.883452,0.606339,1.000000


>각 영화별 평점에 따른 유사도를 dataframe으로 표현해 봤다.

In [13]:
item_based_collab['7번방의 선물'].sort_values(ascending=False)[1:]

moviename
연평대전    0.989949
국제시장    0.874573
명량      0.514496
국가대표    0.000000
Name: 7번방의 선물, dtype: float64

> 7번방의 선물과 연관성이 높은 영화 순으로 나열해 보니 연평대전을 추천해 주고 있다. [1:]을 이용하여 자기 자신을 제외하고 영화를 추천해 주었다.
